In [1]:
import pandas as pd
import torch
import torchaudio
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import MultilabelAccuracy, MultilabelPrecision, MultilabelAveragePrecision
from tqdm import tqdm

# --- Đọc và chuẩn hóa dữ liệu ---
data = pd.read_csv("audio_only_classification_dataset.csv")
print(f"Số mẫu trong CSV: {len(data)}")
data['label'] = data['label'].apply(lambda x: [label.strip() for label in x.split(',')])  # Chuẩn hóa nhãn
print(data['label'].head())

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(data['label'])
label_classes = mlb.classes_
print(f"labels shape: {labels.shape}")
print(f"label classes: {label_classes}")

audio_paths = data['audio_id'].values
train_audio_paths, val_audio_paths, train_labels, val_labels = train_test_split(
    audio_paths, labels, test_size=0.2, random_state=42
)
print(f"train_labels shape: {train_labels.shape}")
print(f"val_labels shape: {val_labels.shape}")
print(f"len(train_audio_paths): {len(train_audio_paths)}")
print(f"len(val_audio_paths): {len(val_audio_paths)}")
print(f"train_labels dtype: {train_labels.dtype}")
print(f"Sample train_labels[0]: {train_labels[0]}")

# Chuyển đổi nhãn sang float32
train_labels = train_labels.astype(np.float32)
val_labels = val_labels.astype(np.float32)
print(f"train_labels dtype after conversion: {train_labels.dtype}")

# --- Tiền xử lý âm thanh ---
def preprocess_audio(audio_path, sample_rate=16000, max_length=5.0):
    waveform, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(sr, sample_rate)
        waveform = resampler(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    max_samples = int(max_length * sample_rate)
    if waveform.shape[1] > max_samples:
        waveform = waveform[:, :max_samples]
    else:
        padding = max_samples - waveform.shape[1]
        waveform = torch.nn.functional.pad(waveform, (0, padding))
    assert waveform.shape == (1, max_samples), f"Waveform shape không đúng: {waveform.shape}"
    return waveform

# --- Transform cho Wav2Vec2 ---
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base')
def transform_waveform(waveform):
    if waveform.ndim == 2:
        waveform = waveform.squeeze(0)
    assert waveform.ndim == 1, f"Waveform phải là 1D, nhận được shape: {waveform.shape}"
    inputs = processor(waveform, sampling_rate=16000, return_tensors='pt', padding=True)
    print(inputs['input_values'].shape)
    input_values = inputs['input_values'].squeeze()
    if input_values.ndim == 2:
        input_values = input_values[0]
    assert input_values.ndim == 1, f"input_values phải là 1D, nhận được shape: {input_values.shape}"
    print(input_values.shape)
    print("-------")
    return input_values

# --- Định nghĩa Dataset ---
class AudioMultilabelDataset(Dataset):
    def __init__(self, audio_paths, labels, transform=None, sample_rate=16000, max_length=5.0):
        self.audio_paths = audio_paths
        self.labels = labels
        self.transform = transform
        self.sample_rate = sample_rate
        self.max_length = max_length
        # print(f"Dataset labels shape: {labels.shape}")
        assert len(audio_paths) == labels.shape[0], f"Số mẫu không khớp: {len(audio_paths)} vs {labels.shape[0]}"
        assert labels.shape[1] == len(label_classes), f"Số nhãn không khớp: {labels.shape[1]} vs {len(label_classes)}"

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        audio_path = self.audio_paths[idx]
        label = self.labels[idx]
        waveform = preprocess_audio(audio_path, self.sample_rate, self.max_length)
        if self.transform:
            waveform = self.transform(waveform)
        
        # print(f"idx: {idx}, waveform shape: {waveform.shape}, label shape: {label.shape}")
        assert waveform.ndim == 1, f"Waveform shape không đúng: {waveform.shape}"
        assert label.shape == (len(label_classes),), f"Label shape không đúng: {label.shape}"
        
        labels_tensor = torch.FloatTensor(label).to(dtype=torch.float32)
        # print(f"Label dtype: {labels_tensor.dtype}")
        return {
            'input_values': waveform,
            'labels': labels_tensor
        }

# --- Tạo dataset ---
train_dataset = AudioMultilabelDataset(train_audio_paths, train_labels, transform=transform_waveform, max_length = 60)
val_dataset = AudioMultilabelDataset(val_audio_paths, val_labels, transform=transform_waveform,max_length = 60)

# --- Collate function ---
def collate_fn(batch):
    input_values = torch.stack([item['input_values'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    # print(f"Batch input_values shape: {input_values.shape}")
    # print(f"Batch labels shape: {labels.shape}")
    assert input_values.shape[0] == len(batch), f"Batch size input_values không đúng: {input_values.shape[0]}"
    assert labels.shape == (len(batch), len(label_classes)), f"Batch labels shape không đúng: {labels.shape}"
    return {
        'input_values': input_values,
        'labels': labels
    }

# --- Tạo DataLoader ---
batch_size = 8  # Có thể thử batch_size=1 để debug
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# --- Định nghĩa mô hình phân loại đa nhãn ---
class MultilabelWav2Vec2(nn.Module):
    def __init__(self, num_labels):
        super(MultilabelWav2Vec2, self).__init__()
        self.wav2vec2 = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base')
        self.classifier = nn.Linear(self.wav2vec2.config.hidden_size, num_labels)
        
    def forward(self, input_values):
        outputs = self.wav2vec2(input_values)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Lấy [CLS] token hoặc mean pooling
        logits = self.classifier(pooled_output)
        return logits

# --- Khởi tạo mô hình, hàm mất mát, optimizer ---

from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultilabelWav2Vec2(num_labels=len(label_classes)).to(device)
criterion = nn.BCEWithLogitsLoss()  # Phù hợp cho phân loại đa nhãn
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.005)

# Thiết lập optimizer với learning rate thích hợp và weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=0.01)

# Thêm learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

# Gradient accumulation cho batch size lớn hơn trên GPU nhỏ
accumulation_steps = 2

# Mixed precision training
scaler = torch.amp.GradScaler(device = device)

d:\projects\v2v\v5\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Số mẫu trong CSV: 8278
0    [1, 7, 8, 10, 11]
1               [8, 9]
2                 [10]
3             [10, 11]
4               [8, 9]
Name: label, dtype: object
labels shape: (8278, 18)
label classes: ['0' '1' '10' '11' '12' '13' '14' '15' '16' '17' '2' '3' '4' '5' '6' '7'
 '8' '9']
train_labels shape: (6622, 18)
val_labels shape: (1656, 18)
len(train_audio_paths): 6622
len(val_audio_paths): 1656
train_labels dtype: int64
Sample train_labels[0]: [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
train_labels dtype after conversion: float32


d:\projects\v2v\v5\.conda\lib\site-packages\transformers\configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [2]:
# --- Hàm tính metrics ---
# def compute_metrics(logits, labels, device):
#     binary_preds = (logits > 0).int()  # Chuyển logits thành nhãn nhị phân (0 hoặc 1)
    
#     logits_tensor = logits.to(device)
#     binary_preds_tensor = binary_preds.to(device)
#     labels_tensor = labels.to(dtype=torch.int32).to(device)  # Chuyển nhãn sang int32
    
#     acc = MultilabelAccuracy(num_labels=labels.shape[1], average='macro').to(device)
#     precision_metric = MultilabelAccuracy(num_labels=labels.shape[1], average='micro').to(device)
#     mAP = MultilabelAveragePrecision(num_labels=labels.shape[1]).to(device)
    
#     acc_value = acc(binary_preds_tensor, labels_tensor).item()
#     precision_value = precision_metric(binary_preds_tensor, labels_tensor).item()
#     map_value = mAP(logits_tensor, labels_tensor).item()  # mAP vẫn dùng logits_tensor
    
#     return {
#         'accuracy': acc_value,
#         'precision': precision_value,
#         'mAP': map_value
#     }

def compute_metrics(logits, labels, device):
    # Sử dụng torch.sigmoid để có giá trị dự đoán tốt hơn
    probs = torch.sigmoid(logits)
    binary_preds = (probs > 0.5).int()
    
    # Chuyển nhãn sang int32 cho torchmetrics
    labels = labels.to(dtype=torch.int32)
    
    # Tính toán metrics một lần trên batch
    acc = MultilabelAccuracy(num_labels=labels.shape[1], average='macro').to(device)
    precision_metric = MultilabelPrecision(num_labels=labels.shape[1], average='micro').to(device)
    mAP = MultilabelAveragePrecision(num_labels=labels.shape[1]).to(device)
    
    acc_value = acc(binary_preds, labels).item()
    precision_value = precision_metric(binary_preds, labels).item()
    map_value = mAP(probs, labels).item()  # Sử dụng probs thay vì logits
    
    return {
        'accuracy': acc_value,
        'precision': precision_value,
        'mAP': map_value
    }



In [3]:
# Early stopping
best_val_map = 0
patience = 1
patience_counter = 0
early_stop = False

num_epochs = 2  # Tăng số epochs
for epoch in range(num_epochs):
    if early_stop:
        print("Early stopping triggered!")
        break
        
    # Training
    model.train()
    train_loss = 0.0
    train_metrics = {'accuracy': 0.0, 'precision': 0.0, 'mAP': 0.0}
    num_batches = 0
    start_time = time.time()
    
    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        input_values = batch['input_values'].to(device)
        labels = batch['labels'].to(device)
        
        # Sử dụng mixed precision
        with torch.amp.autocast('cuda'):
            logits = model(input_values)
            loss = criterion(logits, labels) / accumulation_steps
        
        # Gradient accumulation
        scaler.scale(loss).backward()
        
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            # Gradient clipping để tránh exploding gradients
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        train_loss += loss.item() * accumulation_steps
        
        # Tính metrics chỉ với một tỷ lệ nhất định các batch để tăng tốc
        if i % 5 == 0:  # Tính metrics mỗi 5 batch
            metrics = compute_metrics(logits, labels, device)
            for key in train_metrics:
                train_metrics[key] += metrics[key]
            num_batches += 1
    
    train_loss /= len(train_loader)
    for key in train_metrics:
        train_metrics[key] /= max(num_batches, 1)
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_metrics = {'accuracy': 0.0, 'precision': 0.0, 'mAP': 0.0}
    num_batches = 0
    all_logits = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_values = batch['input_values'].to(device)
            labels = batch['labels'].to(device)
            
            with torch.amp.autocast('cuda'):
                logits = model(input_values)
                loss = criterion(logits, labels)
            
            val_loss += loss.item()
            
            # Thu thập tất cả logits và labels để tính metrics một lần duy nhất
            all_logits.append(logits.detach())
            all_labels.append(labels.detach())
            num_batches += 1
    
    # Tính metrics một lần trên toàn bộ validation set
    all_logits = torch.cat(all_logits, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    val_metrics = compute_metrics(all_logits, all_labels, device)
    
    val_loss /= num_batches
    
    # Cập nhật learning rate dựa trên validation loss
    scheduler.step(val_loss)
    
    # Early stopping check
    if val_metrics['mAP'] > best_val_map:
        best_val_map = val_metrics['mAP']
        patience_counter = 0
        # Lưu model tốt nhất
        torch.save(model.state_dict(), 'best_model.pt')
        print("Saved best model!")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            early_stop = True
    
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs}, Time: {epoch_time:.2f}s, LR: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_metrics['accuracy']:.4f}, "
          f"Train Precision: {train_metrics['precision']:.4f}, Train mAP: {train_metrics['mAP']:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_metrics['accuracy']:.4f}, "
          f"Val Precision: {val_metrics['precision']:.4f}, Val mAP: {val_metrics['mAP']:.4f}")

Epoch 1/2:   0%|          | 0/828 [00:00<?, ?it/s]

torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------
torch.Size([1, 960000])
torch.Size([960000])
-------


Epoch 1/2:   0%|          | 0/828 [00:16<?, ?it/s]


KeyboardInterrupt: 

In [16]:
# --- Vòng huấn luyện ---
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_metrics = {'accuracy': 0.0, 'precision': 0.0, 'mAP': 0.0}
    num_batches = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_values = batch['input_values'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_values)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        metrics = compute_metrics(logits, labels, device)
        for key in train_metrics:
            train_metrics[key] += metrics[key]
        num_batches += 1
    
    train_loss /= num_batches
    for key in train_metrics:
        train_metrics[key] /= num_batches
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
          f"Train Accuracy: {train_metrics['accuracy']:.4f}, "
          f"Train Precision: {train_metrics['precision']:.4f}, "
          f"Train mAP: {train_metrics['mAP']:.4f}")
    
    # Đánh giá trên tập validation
    model.eval()
    val_loss = 0.0
    val_metrics = {'accuracy': 0.0, 'precision': 0.0, 'mAP': 0.0}
    num_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_values = batch['input_values'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_values)
            loss = criterion(logits, labels)
            
            val_loss += loss.item()
            metrics = compute_metrics(logits, labels, device)
            for key in val_metrics:
                val_metrics[key] += metrics[key]
            num_batches += 1
    
    val_loss /= num_batches
    for key in val_metrics:
        val_metrics[key] /= num_batches
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, "
          f"Val Accuracy: {val_metrics['accuracy']:.4f}, "
          f"Val Precision: {val_metrics['precision']:.4f}, "
          f"Val mAP: {val_metrics['mAP']:.4f}")

Epoch 1/5:   0%|          | 0/828 [00:00<?, ?it/s]d:\projects\v2v\v5\.conda\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples found in target, recall is undefined. Setting recall to one for all thresholds.
  warnings.warn(*args, **kwargs)
Epoch 1/5:   2%|▏         | 15/828 [10:26<9:53:06, 43.77s/it]